# test siu nhanh cái model (trong colab)

target: death, injure, total dmg adjust . metric: mse, terminate the large error in predict dmg, phu hop voi gbms

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv('/content/STORM_RAW_1.csv')

In [4]:
df

,Unnamed: 0,ID,EventName,Year,Month,DisasterType,DisasterSubtype,MainLandfallLocation,Flood,Slide,OFDAResponse,Appeal,Declaration,LandfallMagnitude(kph),LandfallPressure(mb),TotalDeaths,NoInjured,TotalDamage(000US$),TotalDamageAdjusted(000US$),CPI
0,0,STORM_NAN_1953,NaN,1953,9.0,Storm,Tropical cyclone,Mekong River Delta,0.0,0.0,0.0,0.0,0.0,93.0,985.0,1000.0,146.0,89730.0,139143.00,9.156133
1,1,STORM_VIOLET_1964,Violet,1964,9.0,Storm,Tropical cyclone,North Central Coast,NaN,NaN,0.0,0.0,0.0,93.0,985.0,124.0,146.0,10000.0,94354.08,10.598376
2,2,STORM_IRIS_1964,Iris,1964,11.0,Storm,Tropical cyclone,South Central Coast,NaN,NaN,0.0,0.0,0.0,150.0,960.0,5100.0,146.0,70000.0,660478.55,10.598376
3,3,STORM_JOAN_1964,Joan,1964,11.0,Storm,Tropical cyclone,South Central Coast,NaN,NaN,0.0,0.0,0.0,130.0,980.0,2500.0,146.0,15000.0,141531.12,10.598376
4,4,STORM_KATE_1964,Kate,1964,11.0,Storm,Tropical cyclone,Central Highlands,NaN,NaN,0.0,0.0,0.0,130.0,970.0,0.0,146.0,3000.0,28306.22,10.598376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,179,STORM_KOMPASU_2021,Kompasu,2021,10.0,Storm,Tropical cyclone,North Central Coast,0.0,0.0,0.0,0.0,0.0,46.0,1002.0,124.0,146.0,89730.0,139143.00,92.590193
180,180,STORM_NAN_2022,NaN,2022,5.0,Storm,Storm (General),NorthEast,1.0,1.0,0.0,0.0,0.0,93.0,985.0,9.0,146.0,89730.0,139143.00,100.000000
181,181,STORM_NORU(KARDING)_2022,Noru(Karding),2022,9.0,Storm,Storm surge,South Central Coast,1.0,0.0,0.0,0.0,0.0,176.0,949.0,9.0,62.0,108300.0,108300.00,100.000000
182,182,STORM_SONCA_2022,Sonca,2022,10.0,Storm,Tropical cyclone,South Central Coast,1.0,1.0,0.0,0.0,0.0,55.0,1000.0,19.0,110.0,65000.0,65000.00,100.000000


In [5]:
# prompt: delete col 	OFDAResponse	Appeal	Declaration and CPI

df = df.drop(['OFDAResponse', 'Appeal', 'Declaration','CPI'], axis=1)


In [6]:
df['Flood'] = df['Flood'].fillna(0)
df['Slide'] = df['Slide'].fillna(0)

## test one hot encoding

In [7]:
# Step 1: Create a mapping for the specific landfall labels
landfall_mapping = {
    'Đông Bắc Bộ': 1,
    'Tây Bắc Bộ': 2,
    'Đồng Bằng Sông Hồng (Hà Nội)': 3,
    'Bắc Trung Bộ': 4,
    'Nam Trung Bộ (Đà Nẵng)': 5,
    'Tây Nguyên': 6,
    'Đông Nam Bộ (TPHCM)': 7,
    'Đồng bằng sông cửu long': 8
}

# Step 2: Map the 'MainLandfallLocation' column to these numerical values
df['MainLandfallLocation'] = df['MainLandfallLocation'].map(landfall_mapping)

# Step 3: Apply one-hot encoding for remaining categorical columns like DisasterType and DisasterSubtype
categorical_cols = ['DisasterType', 'DisasterSubtype']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Step 4: Standardize numerical columns LandfallMagnitude(kph) and LandfallPressure(mb)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['LandfallMagnitude(kph)', 'LandfallPressure(mb)']] = scaler.fit_transform(df[['LandfallMagnitude(kph)', 'LandfallPressure(mb)']])

# Display the processed DataFrame to verify changes
df.head()


,Unnamed: 0,ID,EventName,Year,Month,MainLandfallLocation,Flood,Slide,LandfallMagnitude(kph),LandfallPressure(mb),...,TotalDamage(000US$),TotalDamageAdjusted(000US$),DisasterSubtype_Hail,DisasterSubtype_Lightning/Thunderstorms,DisasterSubtype_Severe weather,DisasterSubtype_Storm (General),DisasterSubtype_Storm surge,DisasterSubtype_Tornado,DisasterSubtype_Tropical cyclone,DisasterSubtype_Tropical depression
0,0,STORM_NAN_1953,NaN,1953,9.0,NaN,0.0,0.0,0.008079,0.009049,...,89730.0,139143.00,False,False,False,False,False,False,True,False
1,1,STORM_VIOLET_1964,Violet,1964,9.0,NaN,0.0,0.0,0.008079,0.009049,...,10000.0,94354.08,False,False,False,False,False,False,True,False
2,2,STORM_IRIS_1964,Iris,1964,11.0,NaN,0.0,0.0,1.606848,-1.656017,...,70000.0,660478.55,False,False,False,False,False,False,True,False
3,3,STORM_JOAN_1964,Joan,1964,11.0,NaN,0.0,0.0,1.045876,-0.323964,...,15000.0,141531.12,False,False,False,False,False,False,True,False
4,4,STORM_KATE_1964,Kate,1964,11.0,NaN,0.0,0.0,1.045876,-0.989991,...,3000.0,28306.22,False,False,False,False,False,False,True,False


## XGBOOST

In [16]:
predictors = ['LandfallMagnitude(kph)', 'LandfallPressure(mb)', 'Flood', 'Slide', 'Year', 'Month'] + \
             [col for col in df.columns if col.startswith('MainLandfallLocation_') or col.startswith('DisasterSubtype_')]

# Exclude highly correlated variables like TotalDamage and focus on TotalDamageAdjusted
target = 'TotalDamageAdjusted(000US$)'

X = df[predictors]
y = df[target]

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(random_state=42)

In [18]:
param_grid = {
    'n_estimators': [100],
    'learning_rate': [0.1],
    'max_depth': [3, 5],
    'subsample': [1.0]
}

# Set up cross-validation with grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)

# Step 4: Train the model using cross-validation to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Step 5: Use the best found model to make predictions
best_model = grid_search.best_estimator_

y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# Step 6: Evaluate the model using MSE for both training and testing sets
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

# Step 7: Analyze feature importance from the best model
feature_importance = best_model.feature_importances_

# Create a DataFrame to display feature importance alongside their names
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

# Output the results: MSE and feature importance
mse_train, mse_test, grid_search.best_params_, importance_df.head(10)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


(4540477968.488963,
 54046626523.95436,
 {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0},
                                 Feature  Importance
 0                LandfallMagnitude(kph)    0.225404
 4                                  Year    0.204793
 2                                 Flood    0.130426
 3                                 Slide    0.093525
 1                  LandfallPressure(mb)    0.093457
 5                                 Month    0.089894
 13  DisasterSubtype_Tropical depression    0.086053
 12     DisasterSubtype_Tropical cyclone    0.049527
 9       DisasterSubtype_Storm (General)    0.026920
 6                  DisasterSubtype_Hail    0.000000)

model overfitting , sẽ sửa ratio train test,  LandfallMagnitude là feature quan trọng nhất, Year cũng có tác động rất lớn vào thiệt hại, flood và slide đồng thời cũng có tác dụng lớn. DisasterSubtype k quan trọng lắm.

## LightGBM , test sau

In [19]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Prepare the LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Set up the parameters for training
params = {
    'objective': 'regression',
    'metric': 'mse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Train the model
num_round = 100
bst = lgb.train(
    params,
    train_data,
    num_boost_round=num_round,
    valid_sets=[train_data, test_data],
    callbacks=[lgb.early_stopping(stopping_rounds=10)]
)

# Make predictions
y_train_pred = bst.predict(X_train, num_iteration=bst.best_iteration)
y_test_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

# Calculate Mean Squared Error (MSE)
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# Display the MSE results
print("Training MSE:", mse_train)
print("Testing MSE:", mse_test)



Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[30]	training's l2: 2.78656e+10	valid_1's l2: 4.74825e+10
Training MSE: 27865598856.255173
Testing MSE: 47482527913.476006


In [20]:
# Feature importance
feature_importance = bst.feature_importance()
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

# Display the top 10 features
importance_df.head(10)

,Feature,Importance
4,Year,27
0,LandfallMagnitude(kph),22
1,LandfallPressure(mb),18
5,Month,18
2,Flood,6
3,Slide,0
6,DisasterSubtype_Hail,0
7,DisasterSubtype_Lightning/Thunderstorms,0
8,DisasterSubtype_Severe weather,0
9,DisasterSubtype_Storm (General),0


đã sửa ratio train test 7-3 và có vẻ fit hơn, nhưng vẫn cần cải thiện dùng regularization như l1 l2 hoặc giảm max_depth
theo model này năm lại là ảnh hưởng nhiều nhất r mới lần lượt tới LandfallMagnitude(kph), LandfallPressure(mb), Month
month thì khá ngạc nhiên, chắc là pattern theo tháng như gió mùa, trời trở,..
khác với model kia thì slide với subtype là 0 quan trọng, sẽ fix one hot encode thành các khu vực . remove subtype chắc hợp lý
